In [ ]:
import c3d
import paths
import opensim as osim
import utils
import numpy as np


In [ ]:
file = paths.USED_MODEL
print(f"Using model: {file}")

In [ ]:
model = osim.Model(file)

In [ ]:
# Get the coordinate set from the model
coordinate_set = model.getCoordinateSet()
all_muscles = model.getMuscles()
state = model.initSystem()  # Initialize the system to get a valid state

# Iterate through each coordinate in the model
for i in range(coordinate_set.getSize()):
    coord = coordinate_set.get(i)
    coord_name = coord.getName()
    range_min = coord.getRangeMin()
    range_max = coord.getRangeMax()
    print(f"Coordinate: {coord_name}, Range: [{range_min}, {range_max}]")
    
    # create numpy list for coordinates with 10 values between min and max
    coord_values = np.linspace(range_min, range_max, 10)
    for j in range(all_muscles.getSize()):
        muscle = all_muscles.get(j)
        muscle_name = muscle.getName()
        
        # compute moment arm for all the points in the coordinate_values
        moment_arms = []
        for value in coord_values:
            coord.setValue(state, value)
            moment_arm = muscle.computeMomentArm(state, coord)
            moment_arms.append(moment_arm)
        
        # if any of the moment arms is > 1e-5, print the muscle and coordinate
        if any(np.abs(moment_arms) > 0.01):
            print(f"Muscle: {muscle_name} has a moment arm for coordinate: {coord_name}")
            
    

In [ ]:
# Get the coordinate set from the model
coordinate_set = model.getCoordinateSet()
all_muscles = model.getMuscles()
state = model.initSystem()  # Initialize the system to get a valid state

# Iterate through each coordinate in the model
for i in range(coordinate_set.getSize()):
    coord = coordinate_set.get(i)
    coord_name = coord.getName()
    range_min = coord.getRangeMin()
    range_max = coord.getRangeMax()
    print(f"\nCoordinate: {coord_name}")

    # Find muscles that actuate this coordinate
    actuating_muscles = []
    for j in range(all_muscles.getSize()):
        muscle = all_muscles.get(j)
        # Check if the moment arm is non-zero for this coordinate
        try:
            moment_arm = muscle.computeMomentArm(state, coord)
            if abs(moment_arm) > 1e-6:
                actuating_muscles.append(muscle.getName())
        except Exception as e:
            pass  # Some muscles may not be able to compute moment arm for some coordinates

    if actuating_muscles:
        print("  Muscles:")
        for m_name in actuating_muscles:
            print(f"    - {m_name}")
    else:
        print("  No muscles actuate this coordinate.")

    

In [ ]:
import opensim as osim

# Load your OpenSim model
model = osim.Model(paths.USED_MODEL)

# Initialize an empty list to store all muscle objects
all_muscles = []

# Get the complete set of forces from the model
force_set = model.getForceSet()

print(f"Scanning {force_set.getSize()} components in the model's ForceSet...")

# Loop through every component in the ForceSet
for i in range(force_set.getSize()):
    # Get the current force component
    force = force_set.get(i)
    
    # Try to cast the force component to the 'Muscle' type
    # This will return a valid muscle object if the cast is successful,
    # and 'None' if it is not a muscle (e.g., a ligament).
    muscle = osim.Muscle.safeDownCast(force)
    
    # If the cast was successful, 'muscle' will not be None
    if muscle:
        all_muscles.append(muscle)

# --- Verification ---
# Print the total number of muscles found
print(f"\n✅ Success! Found a total of {len(all_muscles)} muscles.")

# Print the names of the first 10 muscles found to confirm
print("\n--- Muscle List (first 10) ---")
for muscle in all_muscles[:10]:
    print(f"- {muscle.getName()}")

In [ ]:
import xml.etree.ElementTree as ET

xml_path = r"C:\Git\1_current_projects\powerlifing_model\code\setupFiles\Rajagopal\acquisition.xml"

# Read the XML file
tree = ET.parse(xml_path)
root = tree.getroot()

# Find and print the current laboratory name tag
lab_tag = root.find('.//laboratory')
if lab_tag is not None:
    print("Current laboratory name:", lab_tag.text)
    # Change the laboratory name
    lab_tag.text = "NewLabName"
    print("Laboratory name changed to:", lab_tag.text)
    # Save the changes back to the XML file
    tree.write(xml_path)
else:
    print("Laboratory tag not found.")

In [ ]:
def get_tag_and_subtags(xml_path, tag):
    """
    Returns the text content of the specified tag and its subtags from the XML file.

    Args:
        xml_path (str): Path to the XML file.
        tag (str): Tag to search for (e.g., 'laboratory').

    Returns:
        dict: Dictionary with tag and its subtags as keys and their text as values.
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()
    result = {}

    main_tag = root.find(f'.//{tag}')
    if main_tag is not None:
        result[tag] = main_tag.text
        for sub in main_tag:
            result[sub.tag] = sub.text
    else:
        print(f"Tag <{tag}> not found.")
    return result

# Example usage:
# info = get_tag_and_subtags(xml_path, 'laboratory')
# print(info)

# Visualise 2 trials for comparison

In [ ]:
import paths
print(f"Using model: {paths.USED_MODEL}")